In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
# %qtconsole

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
			
import pandas as pd
import numpy as np
from pathlib import Path		

import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [20]:
df = pd.read_csv(r'data/merged_date_v2.csv', parse_dates = ['count_date'])
df.head()

,count_date,road_name,adt,latitude,longitude,sampling_count
0,1974-07-12,COLWILL RD,125.0,-36.832191,174.636777,3
1,1974-08-11,HOBSONVILLE RD,1900.0,-36.803623,174.638971,22
2,1974-08-11,HOBSONVILLE RD,2853.0,-36.797016,174.650038,1
3,1974-08-11,TAIKATA RD,2490.0,-36.841192,174.651345,1
4,1974-08-11,GLORIA AVE,1823.0,-36.848881,174.650064,5


In [18]:
import re
def add_datepart(df, fldname, drop=True, time=False, errors="raise"):	

    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Dayofweek', 'Dayofyear']
#     'Year', 'Month', 'Week', 'Day',
#             'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [21]:
# add date parts
# df.reset_index(inplace = True)
add_datepart(df, 'count_date')

In [22]:
df['road_name_code'] = df['road_name'].astype('category').cat.codes.astype('category')


In [14]:
df.columns

Index(['road_name', 'adt', 'latitude', 'longitude', 'sampling_count',
       'count_Year', 'count_Month', 'count_Week', 'count_Day',
       'count_Dayofweek', 'count_Dayofyear', 'count_Elapsed',
       'road_name_code'],
      dtype='object')

In [23]:
df.head()

,road_name,adt,latitude,longitude,sampling_count,count_Dayofweek,count_Dayofyear,count_Elapsed,road_name_code
0,COLWILL RD,125.0,-36.832191,174.636777,3,4,193,142819200,774
1,HOBSONVILLE RD,1900.0,-36.803623,174.638971,22,6,223,145411200,1695
2,HOBSONVILLE RD,2853.0,-36.797016,174.650038,1,6,223,145411200,1695
3,TAIKATA RD,2490.0,-36.841192,174.651345,1,6,223,145411200,3628
4,GLORIA AVE,1823.0,-36.848881,174.650064,5,6,223,145411200,1365


In [29]:
xcols = [col for col in df.columns]
xcols.remove('adt')
xcols.remove('road_name')
ycol = 'adt'

In [25]:
# train test split
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
import math

In [26]:
def rmse(x,y): 
    # calculates r^2
    return math.sqrt(np.square(np.subtract(x,y)).mean())
def get_scores(m, X_train, X_valid, y_train, y_valid):
    # returns rmsq training, r^2 validation, avg accuracy training, avg accuracy validation, oob
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): 
        res.append(m.oob_score_)
    return res

def KFold_fit_regression_fit(model, X, y, splits = 5, repeats = 10, random_state = None):
    folds = RepeatedKFold(n_splits = splits, n_repeats = repeats, random_state = random_state)
#     predicted_solution = []
#     correct_solution = []
    results = []
    for train_ind, test_ind in folds.split(X, y):
        X_train, X_test = X.iloc[train_ind,:], X.iloc[test_ind,:]
        y_train, y_test = y[train_ind], y[test_ind]
        model.fit(X_train, y_train)
        r = get_scores(model,X_train, X_test, y_train, y_test)
#         predicted_solution += list(model.predict(X_test))
#         correct_solution += list(y_test)
        results.append(r)
    return np.vstack(results)


# Use oob for model selection and parameter tuning

In [ ]:
xcols = [col for col in df.columns]
xcols.remove('adt')
xcols.remove('road_name')
ycol = 'adt'

In [37]:
# def functions for training
from sklearn.utils import shuffle
def split_vals(df, n):
    return df[:n].copy(), df[n:].copy()

def train_test_split(df, xcols, ycol, train_size_f = 0.75, shuffle_bool = True):
    train_size = int(df.shape[0] * train_size_f)
    if shuffle_bool:
        df_model = shuffle(df)
    X, y = df_model[xcols], df_model[ycol]
    X_train, X_test = split_vals(X, train_size)
    y_train, y_test = split_vals(y, train_size)
    return X_train, X_test, y_train, y_test
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(df, xcols, ycol)
m = RandomForestRegressor(n_estimators=300, max_depth=38, min_samples_leaf = 3, max_features=0.9, oob_score= True, random_state=1, n_jobs  = 3)
m.fit(X_train, y_train)
m.oob_score_
m.score(X_train, y_train)
rf_feat_importance(m, X_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=38,
           max_features=0.9, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=3,
           oob_score=True, random_state=1, verbose=0, warm_start=False)

0.7671413957104946

0.9205339668021487

,cols,imp
1,longitude,0.274751
0,latitude,0.234276
6,road_name_code,0.183604
5,count_Elapsed,0.129411
2,sampling_count,0.104521
4,count_Dayofyear,0.054910
3,count_Dayofweek,0.018527


road_name_code is importance suggesting historical adt is useful

In [45]:
# test the importance of sampling count... turns out it accounts of around 3% r2
xcols.remove('sampling_count')  

In [46]:
X_train, X_test, y_train, y_test = train_test_split(df, xcols, ycol)
m = RandomForestRegressor(n_estimators=300, max_depth=38, min_samples_leaf = 3, max_features=0.9, oob_score= True, random_state=1, n_jobs  = 3)
m.fit(X_train, y_train)
m.oob_score_
m.score(X_train, y_train)
rf_feat_importance(m, X_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=38,
           max_features=0.9, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=3,
           oob_score=True, random_state=1, verbose=0, warm_start=False)

0.7364324736830238

0.9094650460961808

,cols,imp
1,longitude,0.288583
0,latitude,0.256955
5,road_name_code,0.179428
4,count_Elapsed,0.165474
3,count_Dayofyear,0.070973
2,count_Dayofweek,0.038587


In [49]:
xcols.append('sampling_count')

## Try xgboost (to do)

In [ ]:
from xgboost import XGBRegressor
# xgboost cannot handle categorical features


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(df, xcols, ycol)
# m = RandomForestRegressor(n_estimators=300, max_depth=38, min_samples_leaf = 3, max_features=0.9, oob_score= True, random_state=1, n_jobs  = 3)
# m.fit(X_train, y_train)
# m.oob_score_
# m.score(X_train, y_train)
# rf_feat_importance(m, X_train)

# Build a pipeline

In [30]:
from sklearn.utils import shuffle
def split_vals(df, n):
    return df[:n].copy(), df[n:].copy()
train_size = int(df.shape[0]*0.75)
df_model = shuffle(df)
X, y = df_model[xcols], df_model[ycol]
X_train, X_test = split_vals(X, train_size)
y_train, y_test = split_vals(y, train_size)

X_train.head()

,latitude,longitude,sampling_count,count_Dayofweek,count_Dayofyear,count_Elapsed,road_name_code
5887,-36.790134,174.774364,1,5,212,1280534400,424
4867,-36.699220,174.477619,1,0,159,1244419200,2704
7929,-36.750481,174.712383,2,2,53,1329868800,2903
11104,-36.860191,174.622051,3,2,56,1424822400,3124
12088,-36.913403,174.696521,1,1,223,1439251200,3692


In [31]:
m = RandomForestRegressor (n_estimators=150, max_depth=38, min_samples_leaf = 3, max_features=0.9, oob_score= True, random_state=1, n_jobs  = 3)
m.fit(X_train, y_train)
m.oob_score_
m.score(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=38,
           max_features=0.9, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=3,
           oob_score=True, random_state=1, verbose=0, warm_start=False)

0.7687079297634299

0.9218888270673289

In [41]:

imp = rf_feat_importance(m, X_train)
imp

,cols,imp
1,longitude,0.269128
0,latitude,0.239715
6,road_name_code,0.181724
5,count_Elapsed,0.125266
2,sampling_count,0.109020
4,count_Dayofyear,0.056220
3,count_Dayofweek,0.018927


# Feature engineering

- rotate coordinates
- out of fold prediction of adt based on (longitude, latitude, count_dat_columns)
- out of fold sampling count  
- later add 
    - historical maximum adt of the coordinate - impute missing values by out of fold prediction

In [47]:
df.head()

,road_name,adt,latitude,longitude,sampling_count,count_Dayofweek,count_Dayofyear,count_Elapsed,road_name_code
0,COLWILL RD,125.0,-36.832191,174.636777,3,4,193,142819200,774
1,HOBSONVILLE RD,1900.0,-36.803623,174.638971,22,6,223,145411200,1695
2,HOBSONVILLE RD,2853.0,-36.797016,174.650038,1,6,223,145411200,1695
3,TAIKATA RD,2490.0,-36.841192,174.651345,1,6,223,145411200,3628
4,GLORIA AVE,1823.0,-36.848881,174.650064,5,6,223,145411200,1365


## feature engineering - rotate coordinates

In [50]:
X_train, X_test, y_train, y_test = train_test_split(df, xcols, ycol)

In [53]:
from math import cos, sin, radians

In [57]:
def rotate_coordinate(df):
    df['x45'] = df['longitude']*cos(radians(45))
    df['y45'] = df['latitude']*sin(radians(45))


In [59]:
test = X_train.copy()
rotate_coordinate(test)
test.head()

,latitude,longitude,count_Dayofweek,count_Dayofyear,count_Elapsed,road_name_code,sampling_count,x45,y45
14473,-37.006716,174.867709,1,229,1471305600,194,5,123.650143,-26.167700
15303,-36.973956,174.856618,0,326,1479686400,2017,5,123.642301,-26.144535
6032,-36.377890,174.453290,4,253,1284076800,3669,2,123.357104,-25.723053
3061,-36.900546,174.682847,5,72,921283200,3122,1,123.519426,-26.092626
6193,-36.600021,174.651242,2,300,1288137600,3968,1,123.497078,-25.880123


## feature engineering - create grid label

In [74]:
max_lat = df['latitude'].max()
min_lat = df['latitude'].min()
max_long = df['longitude'].max()
min_long = df['longitude'].min()
print(min_lat, max_lat, min_long, max_long)
# one latitude is about 111 kms
# one longitutde is

-57.23805453591322 -4.651237442931668 174.24555715399708 176.1516334349757


In [85]:
lat_steps = np.linspace(min_lat, max_lat, num = 50)
long_steps = np.linspace(min_long, max_long, num = 50)

In [86]:
lat_steps
long_steps

array([-57.23805454, -56.16485419, -55.09165384, -54.01845349,
       -52.94525314, -51.87205279, -50.79885244, -49.72565209,
       -48.65245175, -47.5792514 , -46.50605105, -45.4328507 ,
       -44.35965035, -43.28645   , -42.21324965, -41.1400493 ,
       -40.06684895, -38.99364861, -37.92044826, -36.84724791,
       -35.77404756, -34.70084721, -33.62764686, -32.55444651,
       -31.48124616, -30.40804582, -29.33484547, -28.26164512,
       -27.18844477, -26.11524442, -25.04204407, -23.96884372,
       -22.89564337, -21.82244302, -20.74924268, -19.67604233,
       -18.60284198, -17.52964163, -16.45644128, -15.38324093,
       -14.31004058, -13.23684023, -12.16363988, -11.09043954,
       -10.01723919,  -8.94403884,  -7.87083849,  -6.79763814,
        -5.72443779,  -4.65123744])

array([174.24555715, 174.28445667, 174.32335619, 174.3622557 ,
       174.40115522, 174.44005473, 174.47895425, 174.51785377,
       174.55675328, 174.5956528 , 174.63455231, 174.67345183,
       174.71235135, 174.75125086, 174.79015038, 174.82904989,
       174.86794941, 174.90684892, 174.94574844, 174.98464796,
       175.02354747, 175.06244699, 175.1013465 , 175.14024602,
       175.17914554, 175.21804505, 175.25694457, 175.29584408,
       175.3347436 , 175.37364312, 175.41254263, 175.45144215,
       175.49034166, 175.52924118, 175.5681407 , 175.60704021,
       175.64593973, 175.68483924, 175.72373876, 175.76263828,
       175.80153779, 175.84043731, 175.87933682, 175.91823634,
       175.95713586, 175.99603537, 176.03493489, 176.0738344 ,
       176.11273392, 176.15163343])

In [ ]:
def create_latitude_label(row):
    # say (-47.5792514, 175.21804505)
    for ind, item in enumerate(lat_step):
        label = ind+1
        


def create_longitude_label(row):
    pass
    
    
    
    
#     current_grid_label = 1
#     for lat_ind, item in enumerate(lat_step):
#         if ind == len(lat_stpe):
#             break
#         lat1 = item
#         lat2 = lat_step[lat_ind+1]
#         for long_ind, item2 in enumerate(long_step):
#             if long_ind == len(long_step):
#                 break
#             long1 = item2
#             long2 = long_step[long_ind+1]
#             if row['latitude']

In [ ]:
df['grid_label'] = 

In [71]:
clustering = DBSCAN(eps = 2, min_samples = 3, metric = dbscan_metric).fit(X_train)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [67]:
clustering.labels_.unique()

AttributeError: 'DBSCAN' object has no attribute 'labels_'

In [60]:
df['adt'].std()

8098.809197520319

In [62]:
np.linalg.norm([3,4])

5.0

## feature engineering - distance to the highest adt in a grid

In [ ]:
cos

In [48]:
from mlxtend.regressor import StackingCVRegressor

# generate out of fold prediction of adt
## first generate those for the train dataset

## traom

ModuleNotFoundError: No module named 'mlxtend'